#ImageGeneratorで何をしてるのか見てみよう！

In [0]:
from keras.datasets import cifar10
from keras.preprocessing.image import ImageDataGenerator
import matplotlib.pyplot as plt
import numpy as np
from keras.preprocessing.image import img_to_array

def load():
    (x_train, y_train), (x_test, y_test) = cifar10.load_data()
    return x_test/255

def plot3_3(x):
    plt.figure(figsize=(12,12))
    for i in range(3):
        for j in range(3):
            plt.subplot(3,3,i*3+j+1)
            plt.imshow(x[3*i+j])
    plt.show()

def generator(x):
    NO = 1
    fig = []

    datagen = ImageDataGenerator(rotation_range=20,
                                 width_shift_range=0.2,
                                 height_shift_range=0.2,
                                 horizontal_flip=True)


    for d in datagen.flow(np.expand_dims(x, axis=0), batch_size=1):
        temp = img_to_array(d[0])
        fig.append(temp)
        # datagen.flowは無限ループするため必要な枚数取得できたらループを抜ける
        if NO == 9:
            print("finish")
            break
        NO += 1    

    return fig

In [0]:
test_NO = 3000

data = load()
plt.imshow(data[test_NO])
plt.show()
figs = generator(data[test_NO])
plot3_3(figs)

#GradCAMで可視化してみよう！

In [0]:
#grad-CAM
import keras.backend as K
import cv2
from keras.preprocessing.image import array_to_img

def GradCam(model, x, layer_name):

    X = np.expand_dims(x,axis=0)
    
    # 前処理
    predictions = model.predict(X)
    class_idx = np.argmax(predictions[0])
    class_output = model.output[:, class_idx]
    
    # 勾配を取得
    conv_output = model.get_layer(layer_name).output   # layer_nameのレイヤーのアウトプット
    grads = K.gradients(class_output, conv_output)[0]  # gradients(loss, variables) で、variablesのlossに関しての勾配を返す
    gradient_function = K.function([model.input], [conv_output, grads])  # model.inputを入力すると、conv_outputとgradsを出力する関数

    output, grads_val = gradient_function([X])
    output, grads_val = output[0], grads_val[0]

    # 重みを平均化して、レイヤーのアウトプットに乗じる
    weights = np.mean(grads_val, axis=(0, 1))
    cam = np.dot(output, weights)

    # ヒートマップにして合成
    cam = cv2.resize(cam, (x.shape[1], x.shape[0]), cv2.INTER_LINEAR)
    cam = np.maximum(cam, 0)
    cam = cam / cam.max()
    
    jetcam = cv2.applyColorMap(np.uint8(255 * cam), cv2.COLORMAP_JET)  # モノクロ画像に疑似的に色をつける
    jetcam = cv2.cvtColor(jetcam, cv2.COLOR_BGR2RGB)  # 色をRGBに変換
    jetcam = (np.float32(jetcam) + x*255 / 2)   # もとの画像に合成

    plt.figure(figsize=(8,4))
    plt.subplot(1,2,1)
    plt.imshow(x)
    plt.title("predict:"+labels[class_idx])

    plt.subplot(1,2,2)
    plt.imshow(array_to_img(jetcam))
    plt.title("GradCAM")
    plt.show()

In [0]:
test_NO = 5135
fig = np.expand_dims(X_test[test_NO],axis=0)
img = GradCam(model, fig[0], "activation_4")